In [1]:
"""顧客の退会を予測する"""
#退会してしまう会員が退会する原因を分析する。

#データの読み込み。利用データ整形。
import pandas as pd

customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [2]:
#機械学習用にデータを加工=>未来を予測
#当月と過去1ヶ月の利用回数を集計したデータを作成
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()

/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [3]:
"""退会前月の退会顧客データを作成する"""
#退会前月のデータを作成するのは、退会申請は退会の前月末までに出すことのなっているからである。
#退会した顧客に絞り込んで、end_date列の1ヶ月前の年月を取得し、上記で整形したuselogとcustomer_id,年月をキーにして結合する。
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()


/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

33851


/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#欠損値の除去
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [5]:
"""継続顧客のデータを作成する"""
#継続顧客は、退会月があるわけではないので、どの年月のデータを作成しても良い。
#データを継続顧客に絞り込んだ後、uselogデータに結合して作成する。

conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [6]:
#データをシャッフルして、重複を削除する。
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201807,PL551489,8,5.0,XXXX,C01,M,2018-02-01,NaN,CA1,...,オールタイム,10500.0,通常,6.666667,6.5,11.0,4.0,1.0,2019-04-30,14.0
1,201805,PL746909,3,2.0,XXXXXX,C03,M,2016-11-01,NaN,CA1,...,ナイト,6000.0,通常,4.583333,4.5,8.0,2.0,1.0,2019-04-30,29.0
2,201808,TS622804,4,6.0,XXXX,C02,F,2017-05-01,NaN,CA1,...,デイタイム,7500.0,通常,5.750000,6.0,9.0,4.0,1.0,2019-04-30,23.0
3,201902,GD100850,7,6.0,XXXX,C01,M,2017-02-01,NaN,CA1,...,オールタイム,10500.0,通常,5.166667,5.0,7.0,4.0,1.0,2019-04-30,26.0
4,201807,AS346579,4,3.0,XX,C01,M,2016-02-01,NaN,CA1,...,オールタイム,10500.0,通常,4.666667,4.5,7.0,3.0,1.0,2019-04-30,38.0


In [7]:
#継続顧客と退会顧客の結合
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,name,price,routine_flg,start_date,年月
0,2019-04-30,CA1,通常,C01,オールタイム,8,5.0,PL551489,NaN,NaN,...,11.0,6.666667,6.5,14.0,4.0,XXXX,10500.0,1.0,2018-02-01,201807
1,2019-04-30,CA1,通常,C03,ナイト,3,2.0,PL746909,NaN,NaN,...,8.0,4.583333,4.5,29.0,2.0,XXXXXX,6000.0,1.0,2016-11-01,201805
2,2019-04-30,CA1,通常,C02,デイタイム,4,6.0,TS622804,NaN,NaN,...,9.0,5.750000,6.0,23.0,4.0,XXXX,7500.0,1.0,2017-05-01,201808
3,2019-04-30,CA1,通常,C01,オールタイム,7,6.0,GD100850,NaN,NaN,...,7.0,5.166667,5.0,26.0,4.0,XXXX,10500.0,1.0,2017-02-01,201902
4,2019-04-30,CA1,通常,C01,オールタイム,4,3.0,AS346579,NaN,NaN,...,7.0,4.666667,4.5,38.0,3.0,XX,10500.0,1.0,2016-02-01,201807


In [8]:
"""予測する月の在籍期間を作成"""
#在籍期間のデータを変数として扱う。
#在籍期間の列を追加する。
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

/home/vagrant/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,median,membership_period,min,name,price,routine_flg,start_date,年月,period,now_date
0,2019-04-30,CA1,通常,C01,オールタイム,8,5.0,PL551489,NaN,NaN,...,6.5,14.0,4.0,XXXX,10500.0,1.0,2018-02-01,201807,5,2018-07-01
1,2019-04-30,CA1,通常,C03,ナイト,3,2.0,PL746909,NaN,NaN,...,4.5,29.0,2.0,XXXXXX,6000.0,1.0,2016-11-01,201805,18,2018-05-01
2,2019-04-30,CA1,通常,C02,デイタイム,4,6.0,TS622804,NaN,NaN,...,6.0,23.0,4.0,XXXX,7500.0,1.0,2017-05-01,201808,15,2018-08-01
3,2019-04-30,CA1,通常,C01,オールタイム,7,6.0,GD100850,NaN,NaN,...,5.0,26.0,4.0,XXXX,10500.0,1.0,2017-02-01,201902,24,2019-02-01
4,2019-04-30,CA1,通常,C01,オールタイム,4,3.0,AS346579,NaN,NaN,...,4.5,38.0,3.0,XX,10500.0,1.0,2016-02-01,201807,29,2018-07-01


In [9]:
"""欠損値の除去"""
#機械学習は欠損値があると対応できないので、欠損値は除外するか補間を行う必要がある。
#ここでは、欠損値が含まれているデータの除去を行う。
#最初に、欠損地の数を把握する。
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1               273
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

In [10]:
#欠損値があるend_dateとexit_date,count_1のうち、end_date,exit_dateは退会顧客しか値を持っておらず、継続顧客は欠損値となる。そこで、count_1が欠損しているデータだけ除外する。
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1                 0
customer_id             0
end_date             2621
exit_date            2621
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

In [11]:
"""文字列型の変数を処理できるように整形する"""
#機械学習を行う際に、会員区分や性別などの文字列データにはどのように対応すれば良いか？
#性別などのカテゴリー関連のデータをカテゴリカル変数と呼ぶ。これらの変数も機械学習を行ううえで重要な変数となってくる。これをダミー変数化という。
#ダミー変数化=>

#目的の予測に関するデータに絞り込む。
#ここでは、1ヶ月前の利用回数count_1,カテゴリ変数であるcampaign_name,class_name,gender,定期利用のフラグであるroutine_flg,在籍期間のperiodを説明変数に使用し、目的変数は、退会フラグとなるis_deletedとなる。
#教師あり学習の分類を行う。分類は回帰と違い、退会か継続かの目的変数に用いる。
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,5.0,1.0,5,0.0
1,通常,ナイト,M,2.0,1.0,18,0.0
2,通常,デイタイム,F,6.0,1.0,15,0.0
3,通常,オールタイム,M,6.0,1.0,24,0.0
4,通常,オールタイム,M,3.0,1.0,29,0.0


In [12]:
#データを絞り込んだら、カテゴリカル変数を用いてダミー変数を作成する
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,5,0.0,0,0,1,1,0,0,0,1
1,2.0,1.0,18,0.0,0,0,1,0,0,1,0,1
2,6.0,1.0,15,0.0,0,0,1,0,1,0,1,0
3,6.0,1.0,24,0.0,0,0,1,1,0,0,0,1
4,3.0,1.0,29,0.0,0,0,1,1,0,0,0,1


In [14]:
#pandasは、get_dummiesを使用すると一括でダミー変数化が可能である。文字列データを列に格納すると、簡単にダミー変数化ができる。
#ダミー変数：例えば男性と女性を表現するうえで、0が男性で1が女性であるとき、女性(gender_F)に1が格納されていれば女性で、0であれば男性であると理解できるので、ここでわざわざ男性列(gender_M)を表現する必要はない。会員区分なども同じである。
#ここで、campaign_name_通常,campaign_name_ナイト,gender_M列を削除する。
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,5,0.0,0,0,1,0,0
1,2.0,1.0,18,0.0,0,0,0,0,0
2,6.0,1.0,15,0.0,0,0,0,1,1
3,6.0,1.0,24,0.0,0,0,1,0,0
4,3.0,1.0,29,0.0,0,0,1,0,0


In [15]:
"""決定木を用いて退会予測モデルを作成してみる"""
#退会予測モデルの構築
#決定木アルゴリズムを用いる。=>機械学習によるモデル構築
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1.
 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1.
 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1.

In [16]:
#実際に正解との比較を行うために、実際の値y_testと一緒にデータフレームに格納しておく
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
364,1.0,1.0
2024,0.0,1.0
1895,0.0,0.0
839,1.0,1.0
599,1.0,1.0


In [17]:
"""予測モデルの評価して、モデルのチューニングを行う"""
#先のresults_testデータを集計して正解率を出す。正解しているデータは、results_testデータのy_test列とy_pred列が一致しているデータの件数になる。その件数を、全体のデータ件数で割れば正解率が出る。
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8783269961977186


In [18]:
#機械学習の目的はあくまでも未知のデータへの適合であり、学習用データで予測して精度と評価用データで予測した精度の差が小さいのが理想である。
#score関数を用いて精度を算出する。
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8783269961977186
0.9816223067173637


In [19]:
#決定木モデルの簡易化
#学習用データが評価用データより高い精度を出す場合には、過学習傾向にあるといえる。その場合は、変数の見直しや、モデルのパラメータを変更したりするなどで理想的なモデルに近づける。
#モデルのパラメータをチューニングする
#決定木は、最もキレイに0と1を分割できる説明変数及びその件を探す作業を、木構造上に派生させていく手法である。
#分割していく木構造の深さを浅くしてしまえばモデルは簡易化できる。
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8992395437262357
0.9290240811153359


In [20]:
"""モデルに寄与している変数を確認する"""
#モデルに寄与している変数の確認
importance = pd.DataFrame({"feature_names": X.columns, "coefficient": model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.369413
1,routine_flg,0.130232
2,period,0.500264
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000092
7,gender_F,0.000000


In [22]:
"""顧客の退会を予測"""
#入力はモデル作成時に使用した説明変数となる。
#1ヶ月前の利用回数3回、定期利用者、在籍期間10,キャンペーン区分は入会費無料、会員区分はオールタイム、性別は男性で作成で変数を定義していく。
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

#点数の定義後、データ加工を行う。カテゴリカル変数を用いる。
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]

input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.04347826 0.95652174]]
